In [ ]:
import json
import gzip
from collections import defaultdict
import model

In [ ]:
reviews = []

for file in ["Data/review-SanDiego_10_file1.json.gz", "Data/review-SanDiego_10_file2.json.gz", 
             "Data/review-SanDiego_10_file3.json.gz", "Data/review-SanDiego_10_file4.json.gz"]:
    with gzip.open(file, 'rt', encoding='utf-8') as f:
        for line in f:
            reviews.append(json.loads(line))

meta = {}

with gzip.open("Data/meta-SanDiego.json.gz", 'rt', encoding='utf-8') as f:
    for line in f:
        business = json.loads(line)
        meta[business['gmap_id']] = business

In [ ]:
reviews_valid, meta_valid = model.create_validation_set(reviews, meta, val_size=1000)

In [ ]:
pos, neg  = model.generate_samples(reviews_valid, meta_valid)

pos, neg

In [ ]:
train_samples, test_samples = model.train_test_split(pos, neg, test_size=0.2)

In [ ]:
sorted_businesses = model.baseline_model(train_samples)

In [ ]:
total_revs = sum(count for count, business_id in sorted_businesses)
total_revs

In [ ]:
baseline_preds = model.baseline_predict(sorted_businesses, total_revs)
baseline_preds

In [ ]:
results = model.evaluate_model(test_samples, baseline_preds)
results